# Gauss-Hermite Approximation

### Load packages

In [977]:
from numpy.polynomial.hermite import hermgauss
from matplotlib import pyplot as plt
from numpy.linalg import inv
import pandas as pd
import numpy as np
import scipy

### Randomized simulation data for two sites
- Set random seed
- Generate 10 true betas ranged in (-10, 10)
- Generate 2 sigmas for noise variance in different sites
- Generate data X1 and X2 with size (1000, 10)
- Generate result y1 and y2 with bernoulli distibution 

In [1004]:
np.random.seed(1)
true_beta = (np.random.rand(10,1) - np.random.rand(10,1)) * 10
true_sigma = np.random.rand(2)
X1 = (np.random.rand(1000, 10) - np.random.rand(1000, 10)) * 10
p1 = 1 / (1 + np.exp(-(X1 @ ture_beta + np.random.normal(0, true_sigma[0], 1000).reshape(1000, 1))))
y1 = np.random.binomial(1,p1)
X2 = (np.random.rand(1000, 10) - np.random.rand(1000, 10)) * 10
p2 = 1 / (1 + np.exp(-(X2 @ ture_beta + np.random.normal(0, true_sigma[1], 1000).reshape(1000, 1))))
y2 = np.random.binomial(1,p2)

### Definitions

Notation for $\pi_{ij}$:

$$\pi_{ij} = \dfrac{\exp{(X_{ij}^\top\beta_0}+\mu_{i0})}{1 + \exp{(X_{ij}^\top\beta_0}+\mu_{i0})}$$

In [1133]:
def Pi(x, beta_0, mu):
    return np.asarray((np.exp(x @ beta_0 + mu) / (1 + np.exp(x @ beta_0 + mu))))

In [1134]:
Pi(X1,beta_0,mu[0])

array([[0.23827171],
       [0.86839719],
       [0.22672846],
       [0.03360401],
       [0.35118467],
       [0.5276743 ],
       [0.73545681],
       [0.34497637],
       [0.55633271],
       [0.32880795],
       [0.71327083],
       [0.32250691],
       [0.03534191],
       [0.43652268],
       [0.68122078],
       [0.45443816],
       [0.66688955],
       [0.96454374],
       [0.83142028],
       [0.20353229],
       [0.43480717],
       [0.51772169],
       [0.77405865],
       [0.65409272],
       [0.67937851],
       [0.64298461],
       [0.63699195],
       [0.2549005 ],
       [0.72416997],
       [0.08668989],
       [0.76272899],
       [0.74236071],
       [0.55422946],
       [0.47259504],
       [0.75005874],
       [0.63845609],
       [0.54083444],
       [0.41420659],
       [0.29061305],
       [0.81707249],
       [0.30439803],
       [0.68138722],
       [0.44113256],
       [0.20765672],
       [0.55215873],
       [0.57934645],
       [0.3753603 ],
       [0.837

Notation for $g(\mu_{i0})$:

$$g(\mu_{i0};\beta_0)=\sum_{j=1}^{n_i}\left[y_{ij}\log\pi_{ij}+(1-y_{ij})\log(1-\pi_{ij})\right]+\log\phi(\mu_{i0};\theta_0)$$

In [1154]:
def g(x, y, mu, beta_0, tau=1):
    g = sum(y * np.log(Pi(x, beta_0, mu)) + (1 - y) * np.log(1 - Pi(x, beta_0, mu))) \
    + np.log((np.sqrt(2 * np.pi) * tau)**(-1) * np.exp(-mu**2/(2 * tau**2)))
    return g

In [1155]:
g(X1,y1,mu[0],beta_0)

array([-433.25904471])

Notation for $g'(\mu_{i0})$:

$$\dfrac{\partial g}{\partial \mu_{i0}}=\sum_{j=1}^{n_i}(y_{ij}-\pi_{ij})-\dfrac{\mu_{i0}}{\tau_0^2}$$

In [1156]:
def g_1(x, y, mu, beta_0, tau = 1):
    return sum(y - Pi(x, beta_0, mu)) - mu/tau**2

In [1157]:
g_1(X1,y1,mu[0], beta_0)

array([3.9698119])

Notation for $g''(\mu_{i0})$:

$$\dfrac{\partial^2 g}{\partial \mu_{i0}^2}=-\sum_{j=1}^{n_i}\dfrac{\partial\pi_{ij}}{\partial\mu_{i0}}-\dfrac{1}{\tau_0^2}$$

In [1162]:
def g_2(x, y, mu, beta_0, tau = 1):
    return sum(- np.asarray((np.exp(x @ beta_0 + mu) / (1 + np.exp(x @ beta_0 + mu))**2))) - 1/tau**2

In [1163]:
g_2(X1, y1, mu[0], beta_0)

array([-191.42569678])

Notation for $\pi_{ij}'(\beta_0)$:

$$\dfrac{\partial\pi_{ij}}{\partial\beta_0}=\dfrac{X_{ij}\exp\left(X_{ij}^\top\beta_0+\mu_{i0}\right)}{\left[1+\left(X_{ij}^\top\beta_0+\mu_{i0}\right)\right]^2}$$

In [1164]:
def Pi_1(x, beta_0, mu):
    return np.asarray((np.exp(x @ beta_0 + mu) / (1 + np.exp(x @ beta_0 + mu))**2)) * x

In [1165]:
Pi_1(X1,beta_0,mu[0])

array([[-1.06555402e+00,  6.78885068e-01,  1.26560061e+00, ...,
         4.37692093e-02, -1.42889090e+00, -8.47833265e-01],
       [ 5.49352694e-01, -2.82053675e-01,  3.34459594e-01, ...,
         4.17969535e-01,  3.49631449e-01,  6.23180485e-02],
       [-1.03377693e+00,  1.35575774e-02, -1.28380544e-01, ...,
        -7.02972040e-02, -2.25583185e-01, -4.05229559e-01],
       ...,
       [ 5.89110428e-01,  7.94675227e-01,  2.77164809e-01, ...,
         1.01208415e+00, -8.80752182e-02, -5.10060943e-01],
       [-1.82524965e-01, -1.72130827e-01, -7.58157966e-01, ...,
         9.23939654e-02, -7.87846337e-01, -2.99088135e-01],
       [-2.51295921e-02,  3.36288977e-01,  1.06419395e+00, ...,
         3.46660339e-01, -6.77505495e-05,  1.71275136e+00]])

Notation for $\hat\omega$:

$$\hat\omega=\sqrt{-\dfrac{1}{g''(\hat\mu_{i0})}}$$

In [1166]:
def omega(x, y, mu, beta_0, tau = 1):
    return np.sqrt(-1/g_2(x, y, mu, beta_0))

In [1167]:
omega(X1,y1,mu[0],beta_0)

array([0.07227696])

Notation for Hermite polynomial function $f_k(\hat\mu_{i0};\beta_0)$:

$$h_k\exp\{g(\hat\mu_{i0}+\sqrt{2\pi}\hat\omega x_k;\beta_0)+x_k^2\}$$

In [1168]:
def f_k(k, x, y, mu, beta_0, tau = 1):
    [x_k, h_k] = hermgauss(k)
    return h_k * np.exp(g(x, y, mu + np.sqrt(2 * np.pi) * omega(x, y, mu, beta_0, tau) * x_k, beta_0) + x_k**2)

In [1174]:
f_k(5,X1,y1,mu[0],beta_0)

array([5.48613298e-195, 1.91069329e-190, 6.50970508e-189, 7.53793786e-190,
       9.45357134e-194])

In [1178]:
[x_k, h_k] = hermgauss(5)
np.exp(g(X1, y1, mu[0] + np.sqrt(2 * np.pi) * omega(X1, y1, mu[0], beta_0, tau) * x_k, beta_0))

array([4.64340736e-195, 1.93668163e-190, 6.88632712e-189, 7.64046529e-190,
       8.00140698e-194])

Notation for $\mathcal L_i$:

$$\mathcal L_i=\sqrt{2\pi}\hat\omega\sum_{k=1}^lh_k\exp\left\{g(\hat\mu_{i0}+\sqrt{2\pi}\hat\omega x_k;\beta_0)+x_k^2\right\}$$

In [1179]:
def L(k, x, y, mu, beta_0, tau = 1):
    return np.sqrt(2 * np.pi) * omega(x, y, mu, beta_0) * sum(f_k(k, x, y, mu, beta_0))

In [1180]:
L(5,X1,y1,mu[0],beta_0)

array([1.35057322e-189])

## STEP 1: Maximize $g(\mu_{i0})$

In [1206]:
def max_mu(x, y, mu, beta_0, tau=1, max_iter=100):
    for step in range(max_iter):
#         print('Step: ', step, '\n')
        mu_new = mu - g_1(x, y, mu, beta_0, tau)/g_2(x, y, mu, beta_0, tau)
        diff = mu_new - mu
#         print(diff)
        if diff == 0:
#             print(mu)
            break;
        mu = mu_new
    return mu[0]
#     print('mu converges to:', mu[0], 'in', step + 1, 'steps, with difference', diff[0])

In [1207]:
max_mu(X2, y2, 0, beta_0)

0.21422228255314266

In [1208]:
max_mu(X1, y1, 0, beta_0)

0.024848432558439507

## STEP 2: Maximization preparation of $\beta_0$ in LOCAL

Notation for $\dfrac{\partial\mathcal L_i}{\partial\beta_0}$

$$\dfrac{\partial\mathcal L_i}{\partial\beta_0}=\sqrt{2\pi}\hat\omega\sum_{k=1}^l\left\{f_k(\hat\mu_{i0};\beta_0)h_k\sum_{j=1}^{n_i}(X_{ij}y_{ij}-X_{ij}\pi_{ij})\right\}$$

In [1209]:
def L_1(k, x, y, mu, beta_0, tau = 1):
    L_1 = 0
    [x_k, h_k] = hermgauss(k)    
    for i in range(k):
        L_1 += np.sqrt(2 * np.pi) * omega(x, y, mu, beta_0) *\
        f_k(k, x, y, mu, beta_0, tau = tau)[i] * h_k[i] *\
        np.sum(x * y - x * Pi(x, beta_0, mu + np.sqrt(2 * np.pi) *\
                              omega(x, y, mu, beta_0, tau) * x_k[i]), axis=0)
    return L_1

Notation for $\dfrac{\partial^2\mathcal L_i}{\partial\beta_0^2}$:

$$\dfrac{\partial^2\mathcal L_i}{\partial\beta_0^2}=\sqrt{2\pi}\hat\omega\sum_{k=1}^l\left\{f_k(\hat\mu_{i0};\beta_0)h_k^2\sum_{j=1}^{n_i}(X_{ij}y_{ij}-X_{ij}\pi_{ij})\left[\sum_{j=1}^{n_i}(X_{ij}y_{ij}-X_{ij}\pi_{ij})\right]^\top+f_k(\hat\mu_{i0};\beta_0)h_k\sum_{j=1}^{n_i}\left(-X_{ij}\dfrac{\partial\pi_{ij}}{\partial\beta_0}\right)\right\}$$

In [1210]:
def L_2(k, x, y, mu, beta_0, tau = 1):
    L2 = 0
    [x_k, h_k] = hermgauss(k) 
    for i in range(k):
        boogie = 0
        for j in range(x.shape[0]):
            boogie += -x[j].reshape(x[j].shape[0],1) @\
            Pi_1(x[j], beta_0, mu + np.sqrt(2 * np.pi) *\
                 omega(x, y, mu, beta_0, tau) * x_k[i]).reshape(1,x[j].shape[0])
        L2 += np.sqrt(2 * np.pi) * omega(x, y, mu, beta_0) *\
        (f_k(k, x, y, mu, beta_0, tau = tau)[i] * h_k[i]**2 *\
         np.sum(x * y - x * Pi(x, beta_0, mu + np.sqrt(2 * np.pi) *\
                               omega(x, y, mu, beta_0, tau) * x_k[i]), axis=0).reshape(x.shape[1],1) @\
         np.sum(x * y - x * Pi(x, beta_0, mu + np.sqrt(2 * np.pi) *\
                               omega(x, y, mu, beta_0, tau) * x_k[i]), axis=0).reshape(1,x.shape[1]) +\
        f_k(k, x, y, mu, beta_0, tau = tau)[i] * h_k[i] * boogie)
    return L2

## STEP 3: Maximization of $\beta_0$ in GLOBAL

In [1211]:
def LL_1(site_num, k, X, Y, mu, beta_0, tau = 1):
    LL1 = 0
    for i in range(site_num):
        LL1 += L_1(k, X[i], Y[i], mu[i], beta_0, tau) / L(k, X[i], Y[i], mu[i], beta_0, tau)
    return LL1

In [1212]:
def LL_2(site_num, k, X, Y, mu, beta_0, tau = 1):
    LL2 = 0
    for i in range(site_num):
        LL2 += L_2(k, X[i], Y[i], mu[i], beta_0, tau) / L(k, X[i], Y[i], mu[i], beta_0, tau)
    LL1 = LL_1(site_num, k, X, Y, mu, beta_0, tau)
    return LL2 - LL1.reshape(LL1.shape[0],1) @ LL1.reshape(1,LL1.shape[0])

In [1213]:
def update(site_num, k, X, Y, mu, beta_0, tau = 1):
    direction = LL_1(site_num, k, X, Y, mu, beta_0, tau).reshape(1,beta_0.shape[0]) @\
    inv(LL_2(site_num, k, X, Y, mu, beta_0, tau))
    return direction

# Simulation starts below

### Main function

In [1216]:
beta_0 = np.repeat(0, 10).reshape(10,1)
# betea_0 = true_beta
mu = [0.1, 0.1]
X = [X1, X2]
Y = [y1, y2]
k = 2
tau = 1
print('Initial beta:', beta_0, "\n")
for step in range(10):
    print('Step ', step+1, ':\n')
    for i in range(len(mu)): 
        mu[i] = max_mu(X[i], y[i], mu[i], beta_0)
    for substep in range(100):
        print('Mu:\n', mu, '\n')
        new_beta = beta_0 - update(len(mu), k, X, Y, mu, beta_0).reshape(beta_0.shape[0],1)
        delta = new_beta - beta_0
        beta_0 = new_beta
        print('Beta:\n', beta_0, '\n')
        print('Delta:\n', delta, '\n')

Initial beta: [[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]] 

Step  1 :

Mu:
 [-0.0318751977764525, 0.18779881191730927] 

Beta:
 [[ 1.86405137e-04]
 [ 3.14035802e-04]
 [ 8.01621293e-05]
 [-7.99347068e-05]
 [ 1.89803565e-04]
 [ 1.06191185e-04]
 [ 1.53973269e-04]
 [ 2.76562845e-04]
 [ 4.92968380e-04]
 [ 1.33432008e-04]] 

Delta:
 [[ 1.86405137e-04]
 [ 3.14035802e-04]
 [ 8.01621293e-05]
 [-7.99347068e-05]
 [ 1.89803565e-04]
 [ 1.06191185e-04]
 [ 1.53973269e-04]
 [ 2.76562845e-04]
 [ 4.92968380e-04]
 [ 1.33432008e-04]] 

Mu:
 [-0.0318751977764525, 0.18779881191730927] 

Beta:
 [[ 0.00037304]
 [ 0.00062871]
 [ 0.00016043]
 [-0.00016028]
 [ 0.00037987]
 [ 0.00021262]
 [ 0.00030826]
 [ 0.00055362]
 [ 0.00098721]
 [ 0.0002669 ]] 

Delta:
 [[ 1.86631861e-04]
 [ 3.14677935e-04]
 [ 8.02704966e-05]
 [-8.03437094e-05]
 [ 1.90066389e-04]
 [ 1.06430832e-04]
 [ 1.54290842e-04]
 [ 2.77060830e-04]
 [ 4.94237536e-04]
 [ 1.33466469e-04]] 

Mu:
 [-0.0318751977764525, 0.18779881191730927] 

Beta:
 [[ 

KeyboardInterrupt: 

# Generated the data into files for R

In [981]:
np.savetxt('X1.csv', X1, delimiter=",")
np.savetxt('y1.csv', y1, delimiter=",")
np.savetxt('X2.csv', X2, delimiter=",")
np.savetxt('y2.csv', y2, delimiter=",")

# The followings are NOT true

## Step 1: Combine data to generate golden rules

In [564]:
col = []
for i in range(10): col += ['V'+str(i+1)]
X1 = pd.DataFrame(X1, columns=col)
X2 = pd.DataFrame(X2, columns=col)
# X = pd.concat([X1, X2], ignore_index = True)
y = np.concatenate((y1, y2))

Notation for $\pi_{ij}$:

$$\pi_{ij} = \dfrac{\exp{(X_{ij}^\top\beta_0})}{1 + \exp{(X_{ij}^\top\beta_0})}$$

In [566]:
def Pi(x, beta_0):
    return np.asarray((np.exp(x @ beta_0) / (1 + np.exp(x @ beta_0)))) #need to add /mu_{i0}?

Notation for $L_1$:

$$L_1 = \sum_{j=1}^{n_i}(y_{ij}X_{ij}-\pi_{ij}X_{ij})$$

In [567]:
def L_1(x, y, beta_0):
    return np.expand_dims(np.asarray((y * x - Pi(x, beta_0) * x).sum(axis = 0)), axis=1)

Notation for $L_2$:

$$L_2 = - \sum_{j=1}^{n_i}[\pi_{ij}(1-\pi_{ij})X_{ij}X_{ij}^\top]$$

In [568]:
def L_2(x, y, beta_0):
    XX = 0
    p = Pi(x, beta_0) * (1- Pi(x, beta_0))
    for i in range(x.shape[0]):
#         XX += p[i] * ((np.asarray(x.iloc[i,:]).reshape(x.shape[1],1)))**2 
        XX += p[i] * (np.asarray(x.iloc[i,:]).reshape(x.shape[1],1) \
        @ np.asarray(x.iloc[i,:]).reshape(x.shape[1],1).transpose())
    L2 = -XX
    return L2

In [588]:
beta_0 = np.repeat(0, 10).reshape(10,1)

In [1111]:
print('Initial beta:', beta_0, "\n")
for i in range(100):
    print('Step ', i+1, ':\n')
    beta_0
    beta_new = beta_0 - inv(L_2(X1, y1, beta_0) + L_2(X2, y2, beta_0)) \
    @ (L_1(X1, y1, beta_0) + L_1(X2, y2, beta_0))
    delta = beta_new - beta_0
    beta_0 = beta_new
    print('Beta:\n', beta_0, '\n')
    print('Delta:\n', delta, '\n')
    if (max(np.abs(delta)) == 0):
        break;

Initial beta: [[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]] 

Step  1 :



TypeError: L_2() missing 2 required positional arguments: 'mu' and 'beta_0'

In [591]:
L_2(X1, y1, beta_0)+L_2(X2, y2, beta_0)

array([[-2017.92540591,   278.96089871,    55.13581563,    42.00608349,
           48.96279807,   210.41011784,   147.62714793,   352.95317426,
          260.13291691,    53.38371126],
       [  278.96089871, -1882.57363095,   104.82882103,   -71.34377803,
          332.49827995,   -40.34237896,   356.65541555,   265.13382507,
          523.47528537,   316.84156834],
       [   55.13581563,   104.82882103, -2018.44299488,   -17.02955605,
          -58.167005  ,    96.91290635,   -21.95912822,    81.27766099,
          252.23937134,   165.53816921],
       [   42.00608349,   -71.34377803,   -17.02955605, -1945.83029471,
           63.59659857,    38.07785122,    34.52403033,    49.45718014,
           84.64072536,    18.7808896 ],
       [   48.96279807,   332.49827995,   -58.167005  ,    63.59659857,
        -1959.36670765,    62.41459172,    57.82157479,   327.73823271,
          304.5362868 ,   118.92167227],
       [  210.41011784,   -40.34237896,    96.91290635,    38.07785122,
   

In [550]:
L_2(X1, y1, beta_0)

array([[-4140.35624389],
       [-4107.14077318],
       [-4096.38366046],
       [-4070.5526563 ],
       [-3959.86346214],
       [-4006.78389039],
       [-3930.36488249],
       [-4527.7504776 ],
       [-3984.77596394],
       [-4300.38682221]])

In [565]:
true_beta

array([[-0.0217251 ],
       [ 0.35104993],
       [-2.04337875],
       [-5.75784864],
       [ 1.19368298],
       [-5.78128915],
       [-2.31044591],
       [-2.13129101],
       [ 2.56380536],
       [ 3.40715245]])

In [587]:
inv(L_2(X1, y1, beta_0) + L_2(X2, y2, beta_0)) @ (L_1(X1, y1, beta_0) + L_1(X2, y2, beta_0))

array([[-0.10817893],
       [-0.15577158],
       [-0.04324669],
       [-0.0087236 ],
       [-0.11681485],
       [-0.05317466],
       [-0.07200324],
       [-0.17340093],
       [-0.20606506],
       [-0.11675683]])